In [1]:
# imports
import os
import requests as re
import pandas as pd
from pprint import pprint

In [2]:
# Foursquare API keys
fs_api = os.environ["FS_API_KEY"]
fs_id = os.environ["FS_ID"]
fs_secret = os.environ["FS_SECRET"]

# Yelp API key
yelp_api = os.environ["YELP_API"]

In [3]:
# coordinates for the Calgary Stampede
stamp_coords = "51.0379,-114.0532"

# Yelp

I will set up a function that will retrieve business data from the Yelp API. The function takes in the latitude and longitude coordinates and makes a request to the API using the appropriate API key that was loaded in previously.

In [4]:
def get_yelp(lat_long, query="", radius=1000):
    lat_long = lat_long.split(sep=",")
    
    yelp_url = "https://api.yelp.com/v3/businesses/search"
    
    header = {"Authorization": f"Bearer {yelp_api}"}
    
    yelp_params = {"latitude": lat_long[0],
               "longitude": lat_long[1],
               "radius": radius}
    
    return re.get(yelp_url, params=yelp_params, headers=header).json()

In [5]:
yelp_venues = get_yelp(stamp_coords)
# pprint(yelp_venues)
type(yelp_venues)

dict

The request retrieves a json file in the form of a dictionary. The file contains businesses within a 1000m radius of the Calgary Stampede. I will normalize the data using the key "businesses" to access and explore the nested information.

In [6]:
df_yelp = pd.json_normalize(yelp_venues, record_path=["businesses"])
df_yelp.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,tXoO9Uc_tQ8_IqTmEQcW5w,village-ice-cream-calgary,Village Ice Cream,https://s3-media2.fl.yelpcdn.com/bphoto/t2p0z8...,False,https://www.yelp.com/biz/village-ice-cream-cal...,300,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",4.5,[],...,51.042919,-114.054319,431 10 Avenue SE,,,Calgary,T2G 0W3,CA,AB,"[431 10 Avenue SE, Calgary, AB T2G 0W3, Canada]"
1,SKhV8mF40RcsGrJjkuEwvw,ten-foot-henry-calgary,Ten Foot Henry,https://s3-media1.fl.yelpcdn.com/bphoto/iEGzWX...,False,https://www.yelp.com/biz/ten-foot-henry-calgar...,425,"[{'alias': 'newcanadian', 'title': 'Canadian (...",4.5,[],...,51.041090,-114.065970,1209 1st Street SW,,,Calgary,T2R 0V3,CA,AB,"[1209 1st Street SW, Calgary, AB T2R 0V3, Canada]"
2,vggAigqfXUqSakdVZe3zJw,zcrew-cafe-calgary,ZCREW Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/cD-2Dx...,False,https://www.yelp.com/biz/zcrew-cafe-calgary?ad...,65,"[{'alias': 'cafes', 'title': 'Cafes'}]",4.5,[],...,51.041992,-114.055583,401 11 Avenue SE,,None,Calgary,T2G 0Y5,CA,AB,"[401 11 Avenue SE, Calgary, AB T2G 0Y5, Canada]"
3,kmOeRAvOL1oR6Z-4Ggdx6w,charcut-roast-house-calgary,Charcut Roast House,https://s3-media1.fl.yelpcdn.com/bphoto/gUmXlR...,False,https://www.yelp.com/biz/charcut-roast-house-c...,336,"[{'alias': 'steak', 'title': 'Steakhouses'}]",4.0,[],...,51.044727,-114.063370,101-899 Centre Street S,,,Calgary,T2G 1B8,CA,AB,"[101-899 Centre Street S, Calgary, AB T2G 1B8,..."
4,336_K7i9-jrEhvQleGUVQw,grumans-catering-and-delicatessen-calgary-2,Grumans Catering & Delicatessen,https://s3-media2.fl.yelpcdn.com/bphoto/HYIZld...,False,https://www.yelp.com/biz/grumans-catering-and-...,85,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,[],...,51.042590,-114.059271,230 11th Avenue SE,,,Calgary,T2G 0X8,CA,AB,"[230 11th Avenue SE, Calgary, AB T2G 0X8, Canada]"


In [7]:
# What are the columns?
print(df_yelp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        20 non-null     object 
 1   alias                     20 non-null     object 
 2   name                      20 non-null     object 
 3   image_url                 20 non-null     object 
 4   is_closed                 20 non-null     bool   
 5   url                       20 non-null     object 
 6   review_count              20 non-null     int64  
 7   categories                20 non-null     object 
 8   rating                    20 non-null     float64
 9   transactions              20 non-null     object 
 10  price                     19 non-null     object 
 11  phone                     20 non-null     object 
 12  display_phone             20 non-null     object 
 13  distance                  20 non-null     float64
 14  coordinates.

In [8]:
print(df_yelp.shape)

(20, 24)


The dataframe has 20 rows and 24 columns. Looking at the available columns, I want to extract the necessary information and store them in a new dataframe.

In [31]:
# putting desired columns into a dataframe and renaming some columns
yelp_data = df_yelp[["name", 
                     "review_count", 
                     "rating", 
                     "categories", 
                     "price", 
                     "location.address1",
                     "location.city",
                     "location.zip_code",
                     "coordinates.latitude", 
                     "coordinates.longitude"]].reindex()

yelp_data.rename(columns={"location.address1": "address",
                     "location.city": "city",
                     "location.zip_code": "postal_code",
                     "coordinates.latitude": "latitude",
                     "coordinates.longitude": "longitude"},
                     inplace=True)
yelp_data.head()

,name,review_count,rating,categories,price,address,city,postal_code,latitude,longitude
0,Village Ice Cream,300,4.5,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",$$,431 10 Avenue SE,Calgary,T2G 0W3,51.042919,-114.054319
1,Ten Foot Henry,425,4.5,"[{'alias': 'newcanadian', 'title': 'Canadian (...",$$$,1209 1st Street SW,Calgary,T2R 0V3,51.041090,-114.065970
2,ZCREW Cafe,65,4.5,"[{'alias': 'cafes', 'title': 'Cafes'}]",NaN,401 11 Avenue SE,Calgary,T2G 0Y5,51.041992,-114.055583
3,Charcut Roast House,336,4.0,"[{'alias': 'steak', 'title': 'Steakhouses'}]",$$$,101-899 Centre Street S,Calgary,T2G 1B8,51.044727,-114.063370
4,Grumans Catering & Delicatessen,85,4.5,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",$$,230 11th Avenue SE,Calgary,T2G 0X8,51.042590,-114.059271


Next, I will convert the price column into an integer. [Yelp](https://www.yelp.com/developers/documentation/v3/business_search) uses the dollar($) symbol to denote price levels ranging from 1-4.

In [10]:
yelp_copy = yelp_data.copy()
yelp_copy.loc[yelp_copy["price"] == "$", "price"] = 1
yelp_copy.loc[yelp_copy["price"] == "$$", "price"] = 2
yelp_copy.loc[yelp_copy["price"] == "$$$", "price"] = 3
yelp_copy.loc[yelp_copy["price"] == "$$$$", "price"] = 4  

yelp_copy.rename(columns={"price": "price_level"}, inplace=True)

In [30]:
yelp_copy

,name,review_count,rating,categories,price_level,address,city,postal_code,latitude,longitude
0,Village Ice Cream,300,4.5,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",2,431 10 Avenue SE,Calgary,T2G 0W3,51.042919,-114.054319
1,Ten Foot Henry,425,4.5,"[{'alias': 'newcanadian', 'title': 'Canadian (...",3,1209 1st Street SW,Calgary,T2R 0V3,51.041090,-114.065970
2,ZCREW Cafe,65,4.5,"[{'alias': 'cafes', 'title': 'Cafes'}]",NaN,401 11 Avenue SE,Calgary,T2G 0Y5,51.041992,-114.055583
3,Charcut Roast House,336,4.0,"[{'alias': 'steak', 'title': 'Steakhouses'}]",3,101-899 Centre Street S,Calgary,T2G 1B8,51.044727,-114.063370
4,Grumans Catering & Delicatessen,85,4.5,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",2,230 11th Avenue SE,Calgary,T2G 0X8,51.042590,-114.059271
5,N9NA,136,4.0,"[{'alias': 'newcanadian', 'title': 'Canadian (...",2,121 17th Avenue SE,Calgary,T2G 1H3,51.037710,-114.062240
6,Red's Diner Ramsay,102,4.0,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",2,1101- 8th Street SE,Calgary,T2G 2Z6,51.039960,-114.041900
7,Proof Cocktail Bar,65,4.5,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",2,1302 1st Street SW,Calgary,T2R 0V7,51.040455,-114.065637
8,One18 Empire,87,4.5,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",2,820 Centre Street S,Calgary,T2G 5J2,51.044979,-114.062876
9,Manuel Latruwe,56,4.0,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",3,1333 1 Street SE,Calgary,T2G 5L1,51.040067,-114.061180


In [12]:
yelp_copy.to_excel("yelp_venue_data.xlsx", index=False)

# Foursquare

The Foursquare data extraction process is similar to the one performed for Yelp API.

In [13]:
def get_4sq(lat_long, radius=1000):
    fs_url = "https://api.foursquare.com/v3/places/search"

    params = {
        "radius": 1000, 
        "ll": lat_long}

    headers = {
        "Accept": "application/json",
        "Authorization": fs_api}

    return re.get(fs_url, params=params, headers=headers).json()

In [14]:
fs_venues = get_4sq(stamp_coords)
# pprint(fs_venues)
# type(fs_venues)

In [15]:
df_4sq = pd.json_normalize(fs_venues, record_path="results")
df_4sq.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.country,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,related_places.children,location.cross_street,related_places.parent.fsq_id,related_places.parent.name
0,4b15508cf964a520c3b023e3,"[{'id': 10051, 'name': 'Stadium', 'icon': {'pr...",[],77,/v3/places/4b15508cf964a520c3b023e3,Scotiabank Saddledome,America/Edmonton,51.037431,-114.052107,51.037431,...,CA,"555 Saddledome Rise SE, Calgary AB T2G 2W1",Calgary,[Erlton],T2G 2W1,AB,"[{'name': 'The Club'}, {'fsq_id': '6281b1b26b6...",NaN,NaN,NaN
1,4c07e4e798e69c743fe3c61d,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...",[],67,/v3/places/4c07e4e798e69c743fe3c61d,Nashville North,America/Edmonton,51.034746,-114.055400,51.034746,...,CA,"Stampede Park (Stampede Trail SE), Calgary AB ...",Calgary,[Erlton],T2G 2W1,AB,NaN,Stampede Trail SE,NaN,NaN
2,4b3aa7e2f964a5200a6c25e3,"[{'id': 13022, 'name': 'Sports Bar', 'icon': {...",[],85,/v3/places/4b3aa7e2f964a5200a6c25e3,Dutton's,America/Edmonton,51.037241,-114.053007,51.037241,...,CA,"Olympic Way, Calgary AB",Calgary,NaN,,AB,NaN,,NaN,NaN
3,4b0586ecf964a520d37522e3,"[{'id': 16030, 'name': 'Other Great Outdoors',...",[],243,/v3/places/4b0586ecf964a520d37522e3,Stampede Grounds,America/Edmonton,51.036874,-114.056286,NaN,...,CA,"1410 Olympic Way SE, Calgary AB T2G 2W1",Calgary,NaN,T2G 2W1,AB,"[{'fsq_id': '5d4895a23350fe0008abe1fb', 'name'...",NaN,NaN,NaN
4,518ae730454a31c3d02f473d,"[{'id': 13072, 'name': 'Asian Restaurant', 'ic...",[],322,/v3/places/518ae730454a31c3d02f473d,Zen 8 Grill,America/Edmonton,51.040895,-114.054694,51.040895,...,CA,"412 12th Ave SE (3rd St SE), Calgary AB T2G 1A5",Calgary,[Victoria Park],T2G 1A5,AB,NaN,3rd St SE,NaN,NaN


In [16]:
df_4sq.shape

(10, 22)

In [17]:
df_4sq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   fsq_id                        10 non-null     object 
 1   categories                    10 non-null     object 
 2   chains                        10 non-null     object 
 3   distance                      10 non-null     int64  
 4   link                          10 non-null     object 
 5   name                          10 non-null     object 
 6   timezone                      10 non-null     object 
 7   geocodes.main.latitude        10 non-null     float64
 8   geocodes.main.longitude       10 non-null     float64
 9   geocodes.roof.latitude        9 non-null      float64
 10  geocodes.roof.longitude       9 non-null      float64
 11  location.address              9 non-null      object 
 12  location.country              10 non-null     object 
 13  location

The Foursquare API returned 10 records and 22 columns. Because I will be comparing Foursquare to Yelp, I want to extract information similar to Yelp. From the available columns, these are the name of the business, categories, address, city, zip code, latitude, longitude. Unfortunately, Foursquare does not provide review counts and ratings.

In [28]:
# storing desired data in a dataframe
fs_data = df_4sq[["name",
                  "categories", 
                  "location.address",
                  "location.locality",
                  "location.postcode",
                  "geocodes.main.latitude", 
                  "geocodes.main.longitude"]].reindex()

# renaming certain columns
fs_data.rename(columns={"location.address": "address",
                  "location.locality": "city",
                  "location.postcode": "postal_code",   
                  "geocodes.main.latitude": "latitude",
                  "geocodes.main.longitude": "longitude"},
                  inplace=True)
fs_data

,name,categories,address,city,postal_code,latitude,longitude
0,Scotiabank Saddledome,"[{'id': 10051, 'name': 'Stadium', 'icon': {'pr...",555 Saddledome Rise SE,Calgary,T2G 2W1,51.037431,-114.052107
1,Nashville North,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...",Stampede Park,Calgary,T2G 2W1,51.034746,-114.055400
2,Dutton's,"[{'id': 13022, 'name': 'Sports Bar', 'icon': {...",Olympic Way,Calgary,,51.037241,-114.053007
3,Stampede Grounds,"[{'id': 16030, 'name': 'Other Great Outdoors',...",1410 Olympic Way SE,Calgary,T2G 2W1,51.036874,-114.056286
4,Zen 8 Grill,"[{'id': 13072, 'name': 'Asian Restaurant', 'ic...",412 12th Ave SE,Calgary,T2G 1A5,51.040895,-114.054694
5,Calgary Flames FanAttic,"[{'id': 17043, 'name': 'Clothing Store', 'icon...",555 Saddledome Rise SE,Calgary,T2G 2W1,51.038085,-114.051557
6,Stampede Corral,"[{'id': 18000, 'name': 'Sports and Recreation'...",10 Roundup Way SE,Calgary,T2G 2W1,51.037526,-114.054859
7,Saddleroom Grill,"[{'id': 13049, 'name': 'Diner', 'icon': {'pref...",555 Saddledome Rise SE,Calgary,T2G 2W1,51.037506,-114.051780
8,Stampede Grandstand,"[{'id': 18039, 'name': 'Race Track', 'icon': {...",1410 Olympic Way SE,Calgary,T2G 2W1,51.033675,-114.055181
9,Scotsman Hill,"[{'id': 16032, 'name': 'Park', 'icon': {'prefi...",NaN,Calgary,,51.036773,-114.046421


In [19]:
fs_data.to_excel("foursquare_venue_data.xlsx", index=False)

# Creating a SQLite Database

In [20]:
import sqlite3
from sqlite3 import Error

In [21]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def execute_query(connection, query):
    """executes the query and returns whether or not it was successful"""
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def execute_read_query(connection, query):
    """return results query"""
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [22]:
connection = create_connection("stampede_venues.sqlite")

Connection to SQLite DB successful


In [23]:
# creating tables to store information from the Yelp and Foursquare API
yelp_table = """
CREATE TABLE IF NOT EXISTS yelp_table(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(30) NOT NULL,
    review_count INTEGER,
    rating FLOAT,
    categories VARCHAR(50),
    price_level INTEGER,
    address VARCHAR(30),
    city VARCHAR(30),
    postal_code VARCHAR(10),
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL
);
"""

fs_table = """
CREATE TABLE IF NOT EXISTS fs_table(
    fs_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(30) NOT NULL,
    categories VARCHAR(50),
    address VARCHAR(30),
    city VARCHAR(30),
    postal_code VARCHAR(10),
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL
);
"""

In [24]:
execute_query(connection, yelp_table)
execute_query(connection, fs_table)

Query executed successfully
Query executed successfully


In [25]:
yelp_sql = pd.read_excel("yelp_venue_data.xlsx")
fs_sql = pd.read_excel("foursquare_venue_data.xlsx")

In [26]:
yelp_sql.to_sql("yelp_table", connection, if_exists="replace", index=True)
fs_sql.to_sql("fs_table",connection, if_exists="replace", index=True)

10

# Results

#### Which API has the best coverage?

During the data exploration process, Yelp was able to retrieve 20 venues, while Foursquare retrieved 10. Yelp has the better coverage based on the number of records returned.

#### Top 10 businesses

According to Yelp, the top businesses within 1000m of the Calgary Stampede, based on ratings are:

In [27]:
top10 = """
    SELECT name, rating
    FROM yelp_table
    ORDER BY rating DESC
    LIMIT 10
    """

execute_read_query(connection, top10)

[('Sweet Relief Pastries', 5.0),
 ('Studio Bell', 5.0),
 ('Village Ice Cream', 4.5),
 ('Ten Foot Henry', 4.5),
 ('ZCREW Cafe', 4.5),
 ('Grumans Catering & Delicatessen', 4.5),
 ('Proof Cocktail Bar', 4.5),
 ('One18 Empire', 4.5),
 ('Klein Harris', 4.5),
 ('Rosso Coffee Roasters', 4.5)]

There were no ratings information from the Foursquare API.